In [1]:
#https://svaderia.github.io/articles/downloading-and-unzipping-a-zipfile/
import requests
from urllib.request import urlopen
from tempfile import NamedTemporaryFile
import shutil
from shutil import unpack_archive

import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.windows import Window
from rasterio.mask import mask  

import json

In [10]:
# https://stackoverflow.com/questions/31798194/call-one-function-after-another-python-wrapping
def run_before(lastfunc, *args1, **kwargs1):
    def run(func):
        def wrapped_func(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
            except:
                result = None
            finally:
                lastfunc(*args1, **kwargs1)
                return result
        return wrapped_func
    

    

In [11]:
#delete the folder

def zip_delete(filename):
    file_path = f"/home/giodsubuntu/3D_house_Lidar/Files/{filename}"
    shutil.rmtree(file_path, ignore_errors=True)
    return  

In [12]:
# Open the geotiff file in the previously downloaded folder
all_bounds ={} #where do I put ths to not keep on deleting my keys?

@run_before(zip_delete)
def open_geotiff(filename):
    file_path = f"/home/giodsubuntu/3D_house_Lidar/Files/{filename}/GeoTIFF/{filename}.tif"
    file_num = (file_path.split('/')[-1].replace(".tif", ""))[-2:]

    with rio.open(file_path) as geotiff:
        left, bottom, right, top = geotiff.bounds
        all_bounds[f"{file_num}"] = f"{left, bottom, right, top}"
        with open("/home/giodsubuntu/3D_house_Lidar/json/j_all_bounds.json", "a+") as f:
            json.dump(all_bounds, f)
    return all_bounds

TypeError: 'NoneType' object is not callable

In [9]:
# download and unzip the dtm and dsm files based on URL
@run_before(open_geotiff)
def download_unzip(url):

    with urlopen(url) as zipresp, NamedTemporaryFile() as tfile:
        tfile.write(zipresp.read())
        tfile.seek(0)
        filename = url.split('/')[-1].replace(".zip", "")
        file_path = f"/home/giodsubuntu/3D_house_Lidar/Files/{filename}"
        unpack_archive(tfile.name,file_path, format = 'zip')
        
    return filename

NameError: name 'open_geotiff' is not defined

zipurl_dsm = 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip'
zipurl_dtm = "https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dtm-raster-1m/DHMVIIDTMRAS1m_k03.zip"

url_l = []
for num in range(0,9):
    num +=1
    url=f'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k0{num}.zip'
    url_l.append(url)
for num in range(9,43):
    num +=1
    url=f'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k{num}.zip'
    url_l.append(url)
    
print(url_l)

In [ ]:
url_l = ['https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k02.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k03.zip']

In [ ]:
for url in url_l:
    filename = download_unzip(zipurl_dsm)
    boundaries = open_geotiff(filename)
    zip_delete(filename)

In [ ]:
 #where do I put ths to not keep on deleting my keys?

print(all_bounds)


with open("/home/giodsubuntu/3D_house_Lidar/json/j_all_bounds.json", "w+") as f:
    json.dump(all_bounds, f)


#j_all_bounds = json.dumps(all_bounds)

# the result is a JSON string:
#print(j_all_bounds)
#type(j_all_bounds)

In [ ]:
#DHMVII_vdc_k31all_bounds

#Do you REALLY need to unzip the 2nd file? The Geotiff is there!

#Nice to have: 
# better path for saving file
# handle file already downloaded (chg name?, abort?)